# Library Import

In [1]:
import numpy as np
import pandas as pd
import time
import matplotlib.pyplot as plt

from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC

# Load Dataset

In [2]:
df = pd.read_csv("AirQuality_cleaned.csv", index_col="timestamp", parse_dates=True)

# Preparation

In [3]:
horizons = [1, 6, 12, 24]

# CO Discretization

In [4]:
CO_class_le = LabelEncoder()
CO_class_le.fit(["low", "mid", "high"])

def get_CO_class(CO_val):
    if CO_val < 1.5:
        return "low"
    elif CO_val < 2.5:
        return "mid"
    else:
        return "high"

def get_CO_classes(CO_vals, prefix):
    CO_class = CO_vals.apply(get_CO_class)
    CO_class_l_encoded = CO_class_le.transform(CO_class)
    CO_class_oh_encoded = pd.get_dummies(
        pd.Series(CO_class),
        prefix = prefix,
        dtype = int,
        drop_first = False
    )
    CO_classes = pd.DataFrame({
        prefix + "_class": CO_class,
        prefix + "_class_l_encoded": CO_class_l_encoded
    })

    return pd.concat([CO_classes, CO_class_oh_encoded], axis = 1)

df_temp = df.copy()
CO_classes = get_CO_classes(df_temp["CO(GT)"], "CO")
df_temp = pd.concat([df_temp, CO_classes], axis = 1)

for h in horizons:
    df_temp[f"CO(GT)_fut_{h}h"] = df_temp[f"CO(GT)_fut_{h}h"].fillna(df_temp[f"CO(GT)_fut_{h}h"].mean())
    CO_fut_h_classes = get_CO_classes(df_temp[f"CO(GT)_fut_{h}h"], f"CO_fut_{h}h")
    df_temp = pd.concat([df_temp, CO_fut_h_classes], axis = 1)

pd.set_option('display.max_columns', None)
df_temp.head(5)

,CO(GT),PT08.S1(CO),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T,RH,AH,hour,weekday,month,is_weekend,hour_sin,hour_cos,weekday_sin,weekday_cos,CO(GT)_lag_1h,CO(GT)_lag_6h,CO(GT)_lag_12h,CO(GT)_lag_24h,CO(GT)_ma_6h,CO(GT)_ma_24h,CO(GT)_fut_1h,CO(GT)_fut_6h,CO(GT)_fut_12h,CO(GT)_fut_24h,C6H6(GT)_lag_1h,C6H6(GT)_lag_6h,C6H6(GT)_lag_12h,C6H6(GT)_lag_24h,C6H6(GT)_ma_6h,C6H6(GT)_ma_24h,C6H6(GT)_fut_1h,C6H6(GT)_fut_6h,C6H6(GT)_fut_12h,C6H6(GT)_fut_24h,NOx(GT)_lag_1h,NOx(GT)_lag_6h,NOx(GT)_lag_12h,NOx(GT)_lag_24h,NOx(GT)_ma_6h,NOx(GT)_ma_24h,NOx(GT)_fut_1h,NOx(GT)_fut_6h,NOx(GT)_fut_12h,NOx(GT)_fut_24h,NO2(GT)_lag_1h,NO2(GT)_lag_6h,NO2(GT)_lag_12h,NO2(GT)_lag_24h,NO2(GT)_ma_6h,NO2(GT)_ma_24h,NO2(GT)_fut_1h,NO2(GT)_fut_6h,NO2(GT)_fut_12h,NO2(GT)_fut_24h,CO(GT)_scaled,PT08.S1(CO)_scaled,C6H6(GT)_scaled,PT08.S2(NMHC)_scaled,NOx(GT)_scaled,PT08.S3(NOx)_scaled,NO2(GT)_scaled,PT08.S4(NO2)_scaled,PT08.S5(O3)_scaled,T_scaled,RH_scaled,AH_scaled,hour_scaled,weekday_scaled,month_scaled,is_weekend_scaled,hour_sin_scaled,hour_cos_scaled,weekday_sin_scaled,weekday_cos_scaled,CO(GT)_lag_1h_scaled,CO(GT)_lag_6h_scaled,CO(GT)_lag_12h_scaled,CO(GT)_lag_24h_scaled,CO(GT)_ma_6h_scaled,CO(GT)_ma_24h_scaled,CO(GT)_fut_1h_scaled,CO(GT)_fut_6h_scaled,CO(GT)_fut_12h_scaled,CO(GT)_fut_24h_scaled,C6H6(GT)_lag_1h_scaled,C6H6(GT)_lag_6h_scaled,C6H6(GT)_lag_12h_scaled,C6H6(GT)_lag_24h_scaled,C6H6(GT)_ma_6h_scaled,C6H6(GT)_ma_24h_scaled,C6H6(GT)_fut_1h_scaled,C6H6(GT)_fut_6h_scaled,C6H6(GT)_fut_12h_scaled,C6H6(GT)_fut_24h_scaled,NOx(GT)_lag_1h_scaled,NOx(GT)_lag_6h_scaled,NOx(GT)_lag_12h_scaled,NOx(GT)_lag_24h_scaled,NOx(GT)_ma_6h_scaled,NOx(GT)_ma_24h_scaled,NOx(GT)_fut_1h_scaled,NOx(GT)_fut_6h_scaled,NOx(GT)_fut_12h_scaled,NOx(GT)_fut_24h_scaled,NO2(GT)_lag_1h_scaled,NO2(GT)_lag_6h_scaled,NO2(GT)_lag_12h_scaled,NO2(GT)_lag_24h_scaled,NO2(GT)_ma_6h_scaled,NO2(GT)_ma_24h_scaled,NO2(GT)_fut_1h_scaled,NO2(GT)_fut_6h_scaled,NO2(GT)_fut_12h_scaled,NO2(GT)_fut_24h_scaled,CO_class,CO_class_l_encoded,CO_high,CO_low,CO_mid,CO_fut_1h_class,CO_fut_1h_class_l_encoded,CO_fut_1h_high,CO_fut_1h_low,CO_fut_1h_mid,CO_fut_6h_class,CO_fut_6h_class_l_encoded,CO_fut_6h_high,CO_fut_6h_low,CO_fut_6h_mid,CO_fut_12h_class,CO_fut_12h_class_l_encoded,CO_fut_12h_high,CO_fut_12h_low,CO_fut_12h_mid,CO_fut_24h_class,CO_fut_24h_class_l_encoded,CO_fut_24h_high,CO_fut_24h_low,CO_fut_24h_mid
timestamp,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2004-03-11 18:00:00,4.8,1581.0,20.8,1319.0,281.0,799.0,151.0,2083.0,1409.0,10.3,64.2,0.8065,18,3,3,0,-1.000000,-1.836970e-16,0.433884,-0.900969,2.9,1.6,0.7,2.6,2.816667,1.795833,6.9,1.7,0.8,3.9,11.2,5.2,1.6,11.9,11.316667,6.616667,27.4,5.9,1.8,19.3,243.0,104.0,34.0,166.0,209.000000,125.500000,383.0,122.0,56.0,206.0,135.0,95.0,48.0,113.0,130.500000,97.291667,172.0,97.0,71.0,149.0,2.048974,2.266827,1.475857,1.456457,0.234883,-0.140352,0.878955,1.846083,0.994045,-0.923746,0.880844,-0.550329,0.939133,-0.004969,-0.962879,-0.634869,-1.414701,-0.000222,0.619518,-1.2703,0.612770,-0.369500,-1.049909,0.38555,0.683829,-0.409797,3.636207,-0.294244,-0.974776,1.366759,0.162855,-0.657437,-1.150048,0.257652,0.220967,-0.850995,2.378468,-0.562044,-1.123400,1.268526,0.040106,-0.671967,-1.030845,-0.354230,-0.152250,-0.787145,0.757670,-0.580375,-0.919138,-0.150974,0.515025,-0.394551,-1.464958,0.015406,0.480969,-0.524045,1.356741,-0.349768,-0.942114,0.832018,high,0,1,0,0,high,0,1,0,0,mid,2,0,0,1,low,1,0,1,0,high,0,1,0,0
2004-03-11 19:00:00,6.9,1776.0,27.4,1488.0,383.0,702.0,172.0,2333.0,1704.0,9.7,69.3,0.8319,19,3,3,0,-0.965926,2.588190e-01,0.433884,-0.900969,4.8,1.9,1.1,2.0,3.650000,2.000000,6.1,1.9,1.4,3.7,20.8,7.3,3.2,9.4,14.666667,7.366667,24.0,6.4,4.4,18.2,281.0,146.0,98.0,103.0,248.500000,137.166667,351.0,133.0,109.0,202.0,151.0,112.0,82.0,92.0,140.500000,100.625000,165.0,110.0,104.0,145.0,3.636332,3.182749,2.378559,2.102453,0.757744,-0.525

# Data Splitting

In [5]:
train = df_temp[df_temp.index.year == 2004]
test = df_temp[df_temp.index.year == 2005]

predictors = [
    "PT08.S1(CO)",
    "PT08.S2(NMHC)",
    "PT08.S3(NOx)",
    "PT08.S4(NO2)",
    "PT08.S5(O3)",
    "T",
    "RH",
    "AH",
    "hour",
    "weekday",
    "month",
    "CO(GT)_lag_1h",
    "CO(GT)_lag_6h",
    "CO(GT)_lag_12h",
    "CO(GT)_lag_24h",
    # "CO_lag_1",
    # "CO_lag_6",
    # "CO_lag_12",
    # "CO_lag_24",
    "CO(GT)_ma_1h",
    "CO(GT)_ma_6h",
    "CO(GT)_ma_12h",
    "CO(GT)_ma_24h",
    # "CO_roll_mean_1",
    # "CO_roll_mean_6",
    # "CO_roll_mean_12",
    # "CO_roll_mean_24"
]

scaled_predictors = []

for predictor in predictors:
    scaled_predictors.append(predictor + "_scaled")

# Classification

In [6]:
random_state = 23

def predict_CO_class_using_decision_tree(params, train_df, test_df, h):
    classifier = DecisionTreeClassifier(
        max_depth = params["max_depth"],
        min_samples_leaf = params["min_samples_leaf"],
        max_features = params["max_features"],
        random_state = random_state
    )
    classifier.fit(train_df[scaled_predictors], train_df[f"CO_fut_{h}h_class_l_encoded"])
    return classifier.predict(test_df[scaled_predictors])

def predict_CO_class_using_support_vector_machine(params, train_df, test_df, h):
    classifier = SVC(
        kernel = "rbf",
        C = params["C"],
        gamma = params["gamma"],
        random_state = random_state
    )
    classifier.fit(train_df[scaled_predictors], train_df[f"CO_fut_{h}h_class_l_encoded"])
    return classifier.predict(test_df[scaled_predictors])

def predict_CO_class_using_logistic_regression(params, train_df, test_df, h):
    low_classifier = LogisticRegression(
        penalty = "l2",
        C = params["C"],
        solver = "lbfgs",
        max_iter = 1000,
        random_state = random_state
    )
    low_classifier.fit(train_df[scaled_predictors], train_df[f"CO_fut_{h}h_low"])

    mid_classifier = LogisticRegression(
        penalty = "l2",
        C = params["C"],
        solver = "lbfgs",
        max_iter = 1000,
        random_state = random_state
    )
    mid_classifier.fit(train_df[scaled_predictors], train_df[f"CO_fut_{h}h_mid"])

    high_classifier = LogisticRegression(
        penalty = "l2",
        C = params["C"],
        solver = "lbfgs",
        max_iter = 1000,
        random_state = random_state
    )
    high_classifier.fit(train_df[scaled_predictors], train_df[f"CO_fut_{h}h_high"])

    prob_matrix = np.zeros((len(test_df), 3))
    CO_class_l_encoded = CO_class_le.transform(["low", "mid", "high"])
    prob_matrix[:, CO_class_l_encoded[0]] = low_classifier.predict_proba(test_df[scaled_predictors])[:, 1]
    prob_matrix[:, CO_class_l_encoded[1]] = mid_classifier.predict_proba(test_df[scaled_predictors])[:, 1]
    prob_matrix[:, CO_class_l_encoded[2]] = high_classifier.predict_proba(test_df[scaled_predictors])[:, 1]

    return np.argmax(prob_matrix, axis = 1)

def predict_CO_class_using_naive_baseline(params, train_df, test_df, h):
    return test_df["CO_class_l_encoded"]

predictor_by_clf_name = {
    "Decision Tree": predict_CO_class_using_decision_tree,
    "Support Vector Machine": predict_CO_class_using_support_vector_machine,
    "Logistic Regression": predict_CO_class_using_logistic_regression,
    "Naive Baseline": predict_CO_class_using_naive_baseline
}

params_grids_by_clf_name = {
    "Decision Tree": {
        "max_depth": [1, 3, 5, 7, 9, 11, 13, 15, None],
        "min_samples_leaf": [1, 5, 10, 15, 20, 25],
        "max_features": [None, "sqrt", "log2"],
    },
    "Support Vector Machine": {
        "C": [0.1, 1, 10, 100, 1000],
        "gamma": ["scale", 0.1, 0.01, 0.001, 0.0001],
    },
    "Logistic Regression": {
        "C": [0.01, 0.1, 1, 10, 100, 1000]
    },
    "Naive Baseline": {}
    
}

results = pd.DataFrame({
    "Horizon": horizons,
    "Decision Tree accuracy": [0] * len(horizons),
    "Support Vector Machine accuracy": [0] * len(horizons),
    "Logistic Regression accuracy": [0] * len(horizons),
    "Naive Baseline accuracy": [0] * len(horizons)
})

for h in horizons:
    tuning_train_ratio = 0.8
    tuning_train_test_split = int(len(train) * tuning_train_ratio)
    train_train = train.iloc[:tuning_train_test_split]
    train_test = train.iloc[tuning_train_test_split:]
    y_train_test = train_test[f"CO_fut_{h}h_class_l_encoded"]

    eff_test = test.iloc[:(-1 * h)]
    y_test = eff_test[f"CO_fut_{h}h_class_l_encoded"]

    for clf_name, predict_CO_class in predictor_by_clf_name.items():
        # Hyperparameters tuning
        params_grids = params_grids_by_clf_name[clf_name]
        param_keys = list(params_grids.keys())
        if len(param_keys) > 0:
            params_list = [params_grids[param_keys[0]]]
        else:
            param_list = []
        for i in range(1, len(param_keys)):
            result = []
            next_list = params_grids[param_keys[i]]
            for i in range(0, len(params_list)):
                for j in range(0, len(next_list)):
                    if type(params_list[i]) != list:         
                        params_list[i] = [params_list[i]]
                    temp = [num for num in params_list[i]]
                    temp.append(next_list[j])
                    result.append(temp)
            params_list = result
        params_list = [dict(zip(param_keys, row_list)) for row_list in params_list]
        
        best_params = None
        best_tuning_acc = -np.inf

        print(f"Tuning hyperparameters of {clf_name} for horizon {h}...")
        for curr_params in params_list:
            print("curr_params:", curr_params)
            y_pred_train_test = predict_CO_class(curr_params, train_train, train_test, h)
            curr_tuning_acc = accuracy_score(y_train_test,y_pred_train_test)
            print(clf_name, "with params:", curr_params, f"=> accuracy = {curr_tuning_acc:.4f}")

            if curr_tuning_acc > best_tuning_acc:
                best_tuning_acc = curr_tuning_acc
                best_params = curr_params

        print(f"For horizon: {h}, best params on validation:", best_params, f"with accuracy: {best_tuning_acc:.4f}")
        
        # Train and test the model with the tuned hyperparameters
        print(f"{clf_name} model training for horizon: {h}...")
        y_pred_test = predict_CO_class(best_params, train, eff_test, h)
        test_acc = accuracy_score(y_test, y_pred_test)
        
        print(f"Finished using {clf_name} model for horizon: {h}! Accuracy: {test_acc:.4f}\n\n")

        results.loc[results["Horizon"] == h, clf_name + " accuracy"] = test_acc
        
results

Tuning hyperparameters of Decision Tree for horizon 1...
curr_params: {'max_depth': 1, 'min_samples_leaf': 3, 'max_features': 5}


KeyError: "['CO(GT)_ma_1h_scaled', 'CO(GT)_ma_12h_scaled'] not in index"

# Accuracy Plot

In [ ]:
ax = results.plot(kind = "line", x = "Horizon", figsize=(8, 5))
ax.set_xlabel('Horizon (t + x hours)')
ax.set_ylabel('Accuracy')
ax.set_title('Classification Accuracy by Horizon')
plt.savefig("classification-accuracy.png")
plt.show()
plt.close()

# Export Report

In [ ]:
print("\n=== Exporting Data ===")

# Export to CSV
try:
    csv_filename = "co_class_results.csv"
    results.to_csv(csv_filename, index = False)
    print(f"Successfully saved results to '{csv_filename}'")
except Exception as e:
    print(f"Error saving CSV: {e}")

# Export to Excel
try:
    xlsx_filename = "co_class_results.xlsx"
    results.to_excel(xlsx_filename, index = False)
    print(f"Successfully saved results to '{xlsx_filename}'")
except ImportError:
    print(f"Could not save to Excel ({xlsx_filename}). Missing dependency.\nPlease run: pip install openpyxl")
except Exception as e:
    print(f"Error saving Excel: {e}")